In [ ]:
import numpy as np 
import pandas as pd
import dask

#!pip install dask-ml[complete]
import dask_ml

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# import 
# read input 
# remove columns 
# save the columns in a list
# one hot encode 
# reduce size by int8 etc
# normalize 
# get pca columns 
# train model based on pca columns 
# read test data using the columns list
# one hot encode the columns 
# save the machine id
# normalize the test data
# reduce size by int8 etc
# predict and save and submit

In [ ]:
# import 
import numpy as np 
import pandas as pd 
import gc
import matplotlib.pyplot as plt 
import seaborn as sns 
import sklearn.preprocessing as preprocessing 
from sklearn.model_selection import train_test_split
import dask.dataframe as dd

def mem_usage(pandas_obj):
    if isinstance(pandas_obj,dd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else:
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2
    return "{:03.2f} MB".format(usage_mb)

In [ ]:
# read input 
columns = list(dd.read_csv("../input/trimmeddataset1-1/TrimmedDataset1_1.csv").head())
train_dataset = (dd.read_csv("../input/trimmeddataset1-1/TrimmedDataset1_1.csv", usecols = [i for i in columns if i != 'PuaMode']))

In [ ]:
# remove columns 

# ProductName - yes - 4
train_dataset = train_dataset.drop(columns =  'ProductName')
# EngineVersion - no 65
# AppVersion - no 103
# AvSigVersion - yes - 8422
train_dataset = train_dataset.drop(columns =  'AvSigVersion')
# Platform - no - 4
# Processor - no - 3
# OsVer - yes - 37
train_dataset = train_dataset.drop(columns =  'OsVer')
# OsPlatformSubRelease - no - 9
# OsBuildLab - yes - 484
train_dataset = train_dataset.drop(columns =  'OsBuildLab')
# SkuEdition - no - 8
# Census_MDC2FormFactor - no -12
# Census_DeviceFamily - maybe - 3 with no special distribution
# Census_PrimaryDiskTypeName - maybe with 4 unspecified values
# Census_PowerPlatformRoleName - no - 9
# Census_OsVersion - yes - 362
train_dataset = train_dataset.drop(columns =  'Census_OSVersion')
# Census_OSArchitecture - no -3
# Census_OSBranch - maybe - 27
train_dataset = train_dataset.drop(columns =  'Census_OSBranch')
# Census_OSEdition - maybe - 31
train_dataset = train_dataset.drop(columns =  'Census_OSEdition')
# Census_OSSkuName - maybe - 25
train_dataset = train_dataset.drop(columns =  'Census_OSSkuName')
# Census_OSInstallTypeName - no -9
# Census_OSWUAutoUpdateOptionsName - no - 6
# Census_GenuineStateName - no - 5 - remove outlier 
# train_dataset.drop(rows = train_dataset.index[train_dataset['Census_GenuineStateName']==].tolist(), inplace = True)
# Census_ActivationChannel - no - 6
# Census_FlightRing - no - 9

In [ ]:
categorical_columns = train_dataset.select_dtypes(include=['object']).columns.tolist()
print(categorical_columns)

In [ ]:
columns_to_get_from_test = train_dataset.columns.tolist()
columns_to_get_from_test.remove('HasDetections')

In [ ]:
# one hot encode 
train_dataset = train_dataset.drop(columns ='MachineIdentifier')
train_dataset = train_dataset.categorize()
print(train_dataset.info())

In [ ]:
categorical_columns.remove('MachineIdentifier')
train_dataset = dd.get_dummies(train_dataset,columns = categorical_columns)

In [ ]:
train_dataset.info()

In [ ]:
y_train = train_dataset['HasDetections']
train_dataset = train_dataset.drop(columns='HasDetections')

In [ ]:
test_dataset = pd.read_csv("../input/save-the-attack-contest/test_data.csv", usecols = [i for i in columns_to_get_from_test])

In [ ]:
test_machine_id = test_dataset['MachineIdentifier']
# test_predictions = test_dataset['HasDetections']
test_dataset = test_dataset.drop(columns = 'MachineIdentifier')

In [ ]:
replace = ['0000028988387b115f69f31a3bf04f09', '1.1.15200.1', '4.18.1807.18075', 1.1605880865632915e-06, 6.852675008713112, 0.0170163767268929, 47764.00821294338, 1.3401055691621668, 1.0173610903184032, 0.9984643761631787, 108.39856087077266, 81026.77353313475, 169.56019585788255, 124.22841451232503, 'windows10', 'x64', 15860.682435411201, 584.2029843693655, 'rs4', 'Home', 0.9431323721933259, 2.9843693654484636e-05, 0.0004952713796866335, 118.66845243038371, 0.9798537264845987, 0.9956640847607007, 'Notebook', 'Windows.Desktop', 2178.6814534352998, 237117.55627908043, 3.672241646278447, 4.535354518141476, 2336.6968046779393, 498337.2449334474, 'HDD', 0.07668270765200595, 4940.467389681453, 15.794965755952397, 1485.041267091891, 859.3226920998416, 'Mobile', 'amd64', 15870.64007593562, 1055.4964524967152, 'UUPUpgrade', 14.495754085422258, 59.959899857827956, 'FullAuto', 0.0004798202746448808, 'IS_GENUINE', 'Retail', 'Retail', 413.8543963721495, 33828.562125641845, 0.4917900827748003, 0.0067186988283796534, 0.11884239628280216, 0.029011054601524516, 0.05401278732017979, 0.27186836614374826, 7.83321318963952]

categorical_columns = test_dataset.select_dtypes(include=['object']).columns.tolist()
continuous_columns = test_dataset.drop(columns = categorical_columns)
        
for column in continuous_columns:
        test_dataset[column].fillna(test_dataset[column].mean(), inplace=True)
        

In [ ]:
for column in categorical_columns:
#     if(test_dataset[column].isna().sum() > 0):
        test_dataset[column].fillna(test_dataset[column].value_counts().idxmax(), inplace=True)

In [ ]:
test_dataset.to_csv('test_dataset_filled.csv',index = False)

In [ ]:
test_dataset = dd.read_csv("./test_dataset_filled.csv")

In [ ]:
categorical_columns = test_dataset.select_dtypes(include=['object']).columns.tolist()

In [ ]:
test_dataset = test_dataset.categorize()
test_dataset = dd.get_dummies(test_dataset,columns = categorical_columns)

In [ ]:
train_columns = train_dataset.columns.tolist()
test_columns = test_dataset.columns.tolist()

In [ ]:
for column in train_columns:
    if column not in test_columns:
        train_dataset = train_dataset.drop(columns = column)

In [ ]:
train_columns = train_dataset.columns.tolist()
test_columns = test_dataset.columns.tolist()

In [ ]:
for column in test_columns:
    if column not in train_columns:
        test_dataset = test_dataset.drop(columns = column)

In [ ]:
test_dataset.info()

In [ ]:
train_dataset.info()

In [ ]:
train_dataset = train_dataset[sorted(train_dataset.columns)]
test_dataset = test_dataset[sorted(test_dataset.columns)]

In [ ]:
from dask_ml.preprocessing import StandardScaler

scaler = StandardScaler()
train_dataset = scaler.fit_transform(train_dataset)
test_dataset = scaler.transform(test_dataset)

In [ ]:
from dask_ml.datasets import make_classification

from dask_ml.wrappers import Incremental

from sklearn.neural_network import MLPClassifier

estimator = MLPClassifier(max_iter=5000, hidden_layer_sizes=(100,50,25))

clf = Incremental(estimator)

clf.fit(train_dataset, y_train, classes = [0, 1])

In [ ]:
import dask.bag as db

ypred = clf.predict(test_dataset)
ypred = ypred.compute()

# b = db.from_sequence([{'MachineIdentifier':test_machine_id,'HasDetections':ypred}])

# prediction_df = b.to_dataframe()
# prediction_df.to_csv('test_data_predictions1.csv',single_file = True, index = False)

In [ ]:
prediction_df = pd.DataFrame({'MachineIdentifier':test_machine_id,'HasDetections':ypred})
prediction_df.to_csv('test_data_predictions1.csv',index = False)

xg boost
sgdclassifier
svm 
tree base models